In [1]:
# -*- coding: utf-8 -*-

import os
from PyPDF2 import PdfFileReader, PdfFileWriter
#from nltk.corpus import stopwords 
#from nltk.corpus import stopwords 
#from nltk.tokenize import word_tokenize 
import pandas as pd

import json
import csv

def load_csv(file_name):
    new_dict={}
    with open(file_name,"r",encoding='utf-8') as f:
        #reader=csv.reader(f)
        data=f.read()
        data1=data.split("\n")
        counter=0
        for line in data1:
            if line =="":
                continue
            line1=line.split(",")
            #print(line1)
            if counter==0:
                counter+=1
                continue
            else:
                counter+=1
                ticker=line1[1]
                year=line1[5]
                
                analyst_name=line1[7].lower()
                analyst_name_last = analyst_name.split(" ")[0]
                analyst_code=line1[6]
                company_name=line1[9].lower()
                company_code=line1[8]
                temp_dict={}
                temp_dict['analyst_name'] = analyst_name_last
                temp_dict['analyst_code'] = analyst_code
                temp_dict['company_name'] = company_name
                temp_dict['company_code'] = company_code
                temp_dict['ticker']=ticker
                temp_dict['year']=year
                #print(line1)
                #print(analyst_name_last)
                #print(company_name)
                if year+"|"+ticker not in new_dict.keys():
                    new_dict[year+"|"+ticker] = [temp_dict]
                else:
                    new_dict[year+"|"+ticker] += [temp_dict]
    f.close()
    return new_dict


'''# new_dict=load_csv("D:\\Users\\ly0001\\Desktop\\读研资料\\Module 5\\ECON 6102\\6102_HW3_data\\Analyst List.csv")
# new_dict = load_csv("D:\\Users\\ly0001\\Desktop\\读研资料\\Module 5\\ECON 6102\\6102_HW3_data\\Analyst List.csv", encoding='latin-1')
def load_csv(file_name, encoding='utf-8'):
    new_dict={load_csv("D:\\Users\\ly0001\\Desktop\\读研资料\\Module 5\\ECON 6102\\6102_HW3_data\\Analyst List.csv", encoding='latin-1')}

    with open(file_name, "r", encoding=encoding) as f:
        # Code block to be executed within the 'with' statement
        pass

    with open("analyst_json.txt","w+") as f:
        json.dump(new_dict, f)'''

new_dict=load_csv(r"D:\Users\ly0001\Desktop\读研资料\Module 5\ECON 6102\6102_HW3_data\Analyst List_1.csv")

with open("analyst_json.txt","w+") as f:
    json.dump(new_dict, f)



In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ly0001\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

pdf_read.py

In [8]:
# -*- coding: utf-8 -*-
"""
Spyder Editor

This is a temporary script file.
"""
import os
from PyPDF2 import PdfFileReader, PdfFileWriter
#from nltk.corpus import stopwords 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
stop_words = set(stopwords.words('english')) 
#file_name = 'earning_call.txt'
embedding_path = 'glove.6B.50d.txt'
import json

infn = 'infn.pdf'
outfn = 'outfn.pdf'
# 获取一个 PdfFileReader 对象



def read_pdf(file_name):
    pdf_input = PdfFileReader(open(file_name, 'rb'))
    # 获取 PDF 的页数
    page_count = pdf_input.getNumPages()
    print("page number", page_count)
    # 返回一个 PageObject
    page = pdf_input.getPage(0)
    count=0
    page_content=""
    for page in pdf_input.pages:
        page_content += page.extractText()
    return page_content
        #print(page_content+"\n")



#print(all_files)
def parsing_file_name(str_file):
    file_parsed=str_file.split(" ")
    company_name = file_parsed[0]
    year=file_parsed[1]
    quarter = file_parsed[2] 
    return company_name, year, quarter


##rewrite this session with regexpress matching...
##now we can match it with firm lists and analyst lists...
    
def eliminate_s(sentence, e_list):
    #rint("elist", e_list)
    #print(sentence)
    flag= 0
    while True:
        flag = -1
        for e in e_list:
            if e in sentence:
                flag=1
                loc_=sentence.find(e)
                sentence=sentence[0:loc_]+sentence[loc_+len(e):]
            else:
                pass
        if flag==-1:
            break
    return sentence

def reg_exp_matching(sentence,analyst_this_year_firm):
    #first from
    company=""
    Q_analyst=""
    analyst_list=[]
    analyst_company=[]
    for analyst_info in analyst_this_year_firm:
        if analyst_info['analyst_name']!='' and analyst_info['company_name'] !='':
            analyst_list += [analyst_info['analyst_name'].capitalize()]
            analyst_company +=[analyst_info['company_name'].capitalize()]
    special_company_list = ['J.P. Morgan', 'S.G. Cowen','Robert W. Baird', 'R.W. Baird', 'R. W. Baird','Stifel, Nicolaus']
    eliminate_sequence=[', please proceed', 'ahead', ', proceed', ', sir',' sir']
    reg_pair_head = [ "we have","we will go to","we will go first to","from the line of","line of", " from", "next to", "question for", "line to"]
    reg_pair_tail = ["representing", " with", " at", " from",",", " of"]
    for i in range(len(analyst_list)):
        analyst = analyst_list[i]
        sentence_list=sentence.split(' ')
        if analyst in sentence_list:
            #ok, we find it....
            #find first name...
            for j in range(len(sentence_list)):
                current_word=sentence_list[j]
                if current_word == analyst:
                    if j>0:
                        #find first name? - judege capital letter....
                        if sentence_list[j-1].lower().capitalize() == sentence_list[j-1]: 
                            Q_analyst = sentence_list[j-1]+" "+sentence_list[j]
                        else:
                            Q_analyst = analyst
                    else:
                        Q_analyst = analyst
                else:
                    pass
            company = analyst_company[i]
            print("1",company, Q_analyst)
            return company,Q_analyst
    #discard sequences in eliminate sequences
    #sentence_list=sentence.split(" ")
    #print("sentence", sentence, eliminate_sequence)
    sentence = eliminate_s(sentence, eliminate_sequence)

    
    for head in reg_pair_head:
        index0=sentence.find(head)
        if index0!=-1:
            sentence1 = sentence[index0+len(head):]
            for tail in reg_pair_tail:
                index1=sentence1.find(tail)
                if index1!=-1:
                    temp_list = sentence1[0:index1].split(" ")
                    if len(temp_list)>3:
                        #false finding...
                        continue
                    #find one...
                    if sentence1[0:index1]=="the line ":
                        continue
                    else:
                        Q_analyst = sentence1[0:index1]
                        ##get Q_analyst
                        for spe_company in special_company_list:
                            if spe_company in sentence:
                                company = spe_company
                                return company, Q_analyst
                        dot_pos = sentence1.find(".")
                        if dot_pos == len(sentence1)-1:
                            company = sentence1[index1+len(tail):-1]
                        elif dot_pos ==-1:
                            #not found dot
                            company = sentence1[index1+len(tail):-1]
                        else:
                            #not the last one, but find it...
                            #judge whether the next one is a space
                            if sentence1[dot_pos+1] !=" ":
                                #we are at the end of a sentence
                                company = sentence1[index1+len(tail)+1:dot_pos]
                            else:
                                #count forward until you find the first space...
                                #or perhaps next dot
                                for j in range((len(sentence1)-dot_pos)):
                                    if sentence1[j+dot_pos]==" ":
                                        break
                                company = sentence1[index1+len(tail)+1:(j+dot_pos)]
                else:
                    pass
                if company!="" or Q_analyst!="":
                    break
        else:
            pass
        if company!="" or Q_analyst!="":
            break
    #print("para is\n")
    #print(sentence)
    #print(company)
    #print(Q_analyst)
    if len(company.split(" "))>4 or len(Q_analyst.split(' '))>3:
        print("3")
        return '',''
    names =  Q_analyst.split(' ')
    if len(names)>0:
        last_name = names[-1]
        #check capitalization
        if last_name.lower().capitalize() != last_name:
            print("4",company, Q_analyst)
            return "",""
    print("2",company, Q_analyst)
    return company, Q_analyst


def parse_name_entity(sentence,analyst_info,company,year):
    #name entity recognition...
    #sentence_list = sentence.split(" ")
    #try:
    try:
        total_analysts_this_year_firm = analyst_info[year+"|"+company]
    except:
        total_analysts_this_year_firm = []
    company, name = reg_exp_matching(sentence,total_analysts_this_year_firm)            
    #name=""
    #company=""
    return name, company

#name, company = parse_name_entity("We'll go next to Paul Coster with J.P. Morgan.")
#print(name, company)
#print("extraction complete")
    



def para_extraction(paras,company,year):
    qa_list = []
    count_q = 0
    temp_dic = {}
    para_count=0
    temp_Q_name = ""
    temp_A_name = ""
    temp_Q_agency = ""
    #print("paras is\n", "\n||\n".join(paras))
    with open("analyst_json.txt","r") as f:
        analyst_info = json.load(f)
    
    for para in paras:
        if len(para) == 0:
            continue
        if para[0] is not "<":
            if para_count==0:
                #extract name and agency
                temp_Q_name, temp_Q_agency  = parse_name_entity(para, analyst_info,company,year)
                para_count+=1
            else:
                #find nothing
                continue
        else:
            #print("para is\n", para)
            para_count+=1
            split = para.split('>:')
            if "A" in split[0]:
                session='A'
            elif "Q" in split[0]:
                session='Q'
            else:
                #no-session
                continue
            #session = split[0][-1]
            word_tokens = word_tokenize(split[-1].lower()) 
            #this is a list of words...
            filtered_sentence = [w for w in word_tokens if not w in stop_words]
            if session == 'Q':
                if count_q == 0:
                    #first time
                    temp_dic[session] = filtered_sentence
                    temp_dic["Q_company_name"] = temp_Q_agency
                    temp_dic["Q_analyst_name"] = temp_Q_name
                    temp_dic['A'] = []
                    temp_dic["Manager_name"] = []
                    count_q += 1
                else:
                    #print("temp_dic is\n", temp_dic)
                    if temp_dic["A"] != []:
                        #no answers...
                        qa_list.append(temp_dic)
                    temp_dic = {}
                    temp_dic["Q_company_name"] = temp_Q_agency
                    temp_dic["Q_analyst_name"] = temp_Q_name
                    temp_dic[session] = filtered_sentence
                    temp_dic['A'] = []
                    temp_dic["Manager_name"] = []
            if session == 'A':
                ##get A name...
                index0=split[0].find("-")
                if index0==-1:
                    temp_A_name=""
                else:
                    temp_A = split[0].split("-")
                    temp_A_name= temp_A[-1][1:-1]
                #print("current para is\n", para)
                if "A" not in temp_dic.keys():
                    #do nothing
                    #Q is not found yet...
                    continue
                else:
                    temp_dic['A'].append(filtered_sentence)
                    temp_dic["Manager_name"].append(temp_A_name)
    return qa_list



def clean_data(page_content):
    QA_sessions=page_content.split("Operator")
    para_list=[]
    for session in QA_sessions:
        if "<Q" not in session:
            continue
        session_paragraphs0=session.split("\n")
        temp_sessions=[]
        for para in session_paragraphs0:
            if ("Page" == para[0:4]) or (para==""):
                pass
            else:
                temp_sessions.append(para)
        session1=" ".join(temp_sessions)
        
        #session = session.replace("\n"," ")
        session_paragraphs = session1.split("<")
        all_paras=[]
        for para in session_paragraphs:
            if ">:" in para:
                all_paras.append("<"+para)
            else:
                all_paras.append(para)
        para_list.append(all_paras)
    return para_list

    #print(session)

#page_content=read_pdf("A 2004 Q4 Earnings Call.pdf")

page_content=read_pdf(r"D:\Users\ly0001\Desktop\读研资料\Module 5\ECON 6102\6102_HW3_data\A 2004 Q4 Earnings Call.pdf")

para_list = clean_data(page_content)

#print("\n||\n".join(para_list[0]))

#qa_list_all=[]
#for para in para_list:
#    for para in para_list:
 #       qa_list_temp = para_extraction(para)
  #      qa_list_all.append(qa_list_temp)
    
#print(qa_list_all[0])

#print(para_list)

##all files is the name of list

# 获取一个 PdfFileWriter 对象
#pdf_output = PdfFileWriter()
# 将一个 PageObject 加入到 PdfFileWriter 中
#pdf_output.addPage(page)
# 输出到文件中
#pdf_output.write(open(outfn, 'wb'))


def analyze_all():
    all_files=os.listdir()
    qa_list_all = []
    for file_name in all_files:
        ###establish a record
        qa_file ={}
        if ".pdf" not in file_name:
            continue
        print("now dealing with file", file_name)
        company, year, quarter = parsing_file_name(file_name)
        qa_file["company"]=company
        qa_file["year"]=year
        qa_file["quarter"]=quarter
        temp_content=read_pdf(file_name)
        para_list=clean_data(temp_content)
        qa_file_qas=[]
        for para in para_list:
            qa_list_temp = para_extraction(para,company,year)
            #print("para is\n\n", para)
            qa_file_qas.append(qa_list_temp)
        qa_file["QA_session"] = qa_file_qas
        qa_list_all.append(qa_file)
    return qa_list_all
        ####construct data/

qa_list_all = analyze_all()

#print(qa_list_all[0])
with open('qa_list_all.txt', 'w') as outfile:
    json.dump(qa_list_all, outfile)


<>:211: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:211: SyntaxWarning: "is not" with a literal. Did you mean "!="?
C:\Users\ly0001\AppData\Local\Temp\ipykernel_1856\3089858056.py:211: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if para[0] is not "<":


page number 14
now dealing with file A 2003 Q1 Earnings Call.pdf
page number 13
1 Lehman Edward White
2 Goldman Sachs.  Dean Gray
1 Wrhco Jerry Fleming
2  
2 SG Cowen.  Richard Shoe
2  
2  
2 [indiscernible] Capital.  Alec Berming
2  
2 [indiscernible] Capital.  Alec Berming
2 Thomas Weisel Partners.  Paul [indiscernible]
2  
2  
2  
2  
2 Goldman Sachs.  Dean Gray
2  
2 Lehmann Brothers.  Edward Lang
now dealing with file A 2003 Q2 Earnings Call.pdf
page number 12
2 Morgan Stanley.  Arindam Basu
1 Friedman Deane Dray
1 Wheat Stephen Koffler
1 Fboston Max Schuetz
2 Lehman Brothers.  Adam Wolfman
1 Weisel Paul Knight
2 SG Cowen.  Richard Chu
2 Nomura Securities.  Shailesh Jaitly
2 Lehman Brothers.  Adam Wolfman
2 Morgan Stanley.  Arindam Basu
now dealing with file A 2003 Q4 Earnings Call.pdf
page number 13
2 Goldman Sachs.  Deane Dray
2 SG Cowen.  Richard Chu
1 Lehman Edward White
2 JP Morgan.  Paul Coster
2 Morgan Stanley.  Arindam Basu
1 Stifel Ajit Pai
2  
1 Montsec Mark Fitzgerald
2

C:\Users\ly0001\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\PyPDF2\_cmap.py:142: PdfReadWarning: Advanced encoding /ZapfDingbatsEncoding not implemented yet
  warnings.warn(


1 Friedman Deane Dray
1 Fboston William Stein
1 Montsec Jon Wood
1 Macquari Jon Groberg
1 Stifel Ajit Pai
2 Robert Baird.  Richard Eastman
2 Leerink Swann.  Isaac Ro
1 Fboston William Stein
now dealing with file A 2009 Q4 Earnings Call.pdf
page number 16
1 Montsec Jon Wood
1 Fboston William Stein
1 Friedman Deane Dray
1 Macquari Jon Groberg
1 Longbow Mark Douglass
2 Thomas Weisel Partners.  Savanne Ema
now dealing with file A 2010 Q1 Earnings Call.pdf
page number 16
1 Friedman Deane Dray
1 Fboston William Stein
1 Macquari Jon Groberg
4 Mr.  the line
1 Montsec Jon Wood
1 Lehman Tony Butler
1 Longbow Mark Douglass
1 Stifel Ajit Pai
now dealing with file A 2010 Q2 Earnings Call.pdf
page number 13
1 Fboston William Stein
1 Longbow Mark Douglass
1 Montsec Jon Wood
1 Macquari Jon Groberg
1 Stifel Ajit Pai
1 Fboston William Stein
now dealing with file A 2010 Q3 Earnings Call.pdf
page number 16
1 Montsec Jon Wood
1 Fboston William Stein
1 Macquari Jon Groberg
1 Lehman Tony Butler
1 Longbow Mar

earningcall.py

In [10]:
# -*- coding: utf-8 -*-

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import json
stop_words = set(stopwords.words('english')) 
file_name = 'earning_call.txt'
embedding_path = 'glove.6B.50d.txt'

# TODO: 去标点
def read_file(file_name):
    qa_list = []
    with open(file_name, 'r') as f:
        count_q = 0
        temp_dic = {}
        for line in f:
            if line[0] is not '<':
                continue
            split = line.split('>:')
            session = split[0][-1]
            word_tokens = word_tokenize(split[-1].lower()) 
            filtered_sentence = [w for w in word_tokens if not w in stop_words]
            if session == 'Q':
                if count_q == 0:
                    temp_dic[session] = filtered_sentence
                    temp_dic['A'] = []
                    count_q += 1
                else:
                    qa_list.append(temp_dic)
                    temp_dic = {}
                    temp_dic[session] = filtered_sentence
                    temp_dic['A'] = []
            if session == 'A':
                temp_dic[session].append(filtered_sentence)
    return qa_list

#qa = read_file(file_name)


#print("company is\n", qa_all[0]["company"],qa_all[0]["year"],qa_all[0]["quarter"])
#print("qa is\n",qa[0][0])

import numpy as np

def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile,'r',encoding='UTF-8')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print("Done.",len(model)," words loaded!")
    return model

glove = loadGloveModel(embedding_path)

from scipy.spatial.distance import cosine

def get_embedding(sentence, glove):
    length = len(sentence)
    embedding = 0
    for w in sentence:
        if w in glove:
            embedding += glove[w]
        else:
            length -= 1
    embedding /= length
    return embedding

def get_similarity(qas, glove):
    results = []
    for qa in qas:
        temp_res = []
        q_embedding = get_embedding(qa['Q'], glove)
        for a in qa['A']:
            a_embedding = get_embedding(a, glove)
            temp_res.append(cosine(q_embedding, a_embedding))
        results.append(temp_res)
    return results

def get_similarity1(qa, glove):
    #results = []
    temp_res = []
    try:
        q_embedding = get_embedding(qa['Q'], glove)
    except:
        return [0]
    for a in qa['A']:
        try:
            a_embedding = get_embedding(a, glove)
            temp_res.append(cosine(q_embedding, a_embedding))
        except:
            continue
    #results.append(temp_res)
    if temp_res==[]:
        return [0]
    return temp_res

def get_correlation_answer(qa, glove):
    answers = qa["A"]
    #cur_embedding = 0.0
    temp_res=[]
    sen_list=[]
    if len(answers)==1:
        n_answers=1
        para_cor=0
        count=0
        for a in qa["A"]:
            if a==[]:
                continue
            sen_temp = " ".join(a)
            sen_list+= sen_temp.split(".")
            #count=count+1
            #if count==1:
            #    a_embedding = get_embedding(a, glove)
                
            #else:
            #    cur_embedding=a_embedding
            #    a_embedding = get_embedding(a, glove)
            #    temp_res.append(cosine(cur_embedding, a_embedding))
    else:
        count=0
        for a in qa["A"]:
            if a==[]:
                continue
            sen_temp = " ".join(a)
            sen_list+= sen_temp.split(".")
            count=count+1
            if count==1:
                try:
                    a_embedding = get_embedding(a, glove)
                except:
                    continue
            else:
                try:
                    cur_embedding=a_embedding
                    a_embedding = get_embedding(a, glove)
                    temp_res.append(cosine(cur_embedding, a_embedding))
                except:
                    continue
        n_answers=len(answers)
        if temp_res==[]:
            para_cor = 0
        else:
            para_cor = np.mean(temp_res)
    #now compute for sen list...
    #print("sen_list is")
    #print(sen_list)
    sen_list1=[]
    for sen in sen_list:
        sen1=sen.split(" ")
        #print("sen1 is")
        #print(sen1)
        temp_sen=[]
        if len(sen1)==0:
            pass
        else:
            count_non_trivial=0
            for item in sen1:
                if item!="":
                    count_non_trivial+=1
                    temp_sen+=[item]
            if count_non_trivial ==0 :
                pass
            else:
                sen_list1 +=[temp_sen]
    sen_list=sen_list1
    count = 0
    if len(sen_list)==1:
        return 1,0,1,0
    temp_res_sen=[]
    for sen in sen_list:
        count=count+1
        if count == 1:
            new_sen = sen
        else:
            cur_sen=new_sen
            new_sen = sen
            #print("cur_sen", cur_sen)
            #print("new_sen", new_sen)
            if cur_sen == [] or cur_sen==[""]:
                continue
            if new_sen == [] or new_sen==[""]:
                continue
            try:
                cur_embedding = get_embedding(cur_sen, glove)
                new_embedding = get_embedding(new_sen, glove)
                temp_res_sen.append(cosine(cur_embedding, new_embedding))
            except:
                temp_res_sen.append(0)
    if temp_res_sen==[]:
        sen_cor=0
    else:
        sen_cor=np.mean(temp_res_sen)
    return n_answers, para_cor, len(sen_list1), sen_cor

#results = get_similarity(qa[0],glove)
#print(results)
    

def get_summary(qa):
    #results = []
    #temp_res = []
    #q_embedding = get_embedding(qa['Q'], glove)
    eff_q_words = len(qa['Q'])
    eff_a_words = 0
    n_sentence=0
    for a in qa['A']:
        #a_embedding = get_embedding(a, glove)
        #temp_res.append(cosine(q_embedding, a_embedding))
        eff_a_words+=len(a)
        sen = " ".join(a)
        n_sentence += len(sen.split("."))
    #results.append(temp_res)
    return eff_q_words,eff_a_words, n_sentence

    

def produce_all(qa_all,glove):
    file_str = "company\tyear\tquarter\tQ_company\tQ_analyst\tsession_number\tQA_similarity\tN_A_paragraphs\tsub_session_number\tManager_Name\tN_A_sentences\tN_A_para_corr\tN_A_sen_corr\tN_Q_words\tN_A_words\n".encode("utf-8")
    session_number = 0
    subsession_number = 0
    count_num=0
    for earning_report in qa_all:
        count_num+=1
        print("now deal with", count_num)
        temp_company = earning_report["company"]
        temp_year=earning_report["year"]
        temp_quarter=earning_report["quarter"]
        for qa in earning_report["QA_session"]:
            session_number+=1
            subsession_number = 0
            for session in qa:
                subsession_number+=1
                #this is a qa-session that contains multiple qas
                #we can get a list of paras
                #each session has multiple Q-A rounds...
                temp_Q_company = session["Q_company_name"]
                temp_Q_analyst = session["Q_analyst_name"]
                #temp_sentence_Q= session["Q"]
                #temp_sentence_list_A=session["A"]
                ##each round creates a record?
                #print("session is\n", session)
                result = get_similarity1(session,glove)
                if result == [0]:
                    continue
                record_temp = [temp_company, temp_year, temp_quarter, temp_Q_company, temp_Q_analyst,str(session_number)]
                #calculate the mean of QA...
                #subsession_number = 0
                #for sub_result in results:
                #subsession_number +=1
                mean_cor=np.mean(result)
                n_answers,n_cor, n_sentence, sen_cor=get_correlation_answer(session,glove)
                N_A_paragraphs = n_answers
                eff_q_words, eff_a_words, n_sentence1 = get_summary(session)
                    #I did not count number of words...
                try:
                    temp_manager = session["Manager_name"][0]
                except:
                    temp_manager = ""
                #print("Manager name is\n", session["Manager_name"])
                temp_new_result = "\t".join(record_temp)+"\t"+str(mean_cor)[0:4]+"\t"+str(N_A_paragraphs)+"\t"+str(subsession_number)+"\t"+temp_manager+"\t"+str(n_sentence)+"\t"+str(n_cor)+"\t"+str(sen_cor)+"\t"+str(eff_q_words)+"\t"+str(eff_a_words)+"\n"
                file_str = file_str+temp_new_result.encode("utf-8")
                #print("total file is\n")
                #print(file_str)
    #print(type(file_str))
    #print(len(file_str))
    file_str1=file_str.decode("utf-8")
    #print(type(file_str1))
    with open("result_table1.csv", "w", encoding="utf-8") as f:
        f.write(file_str1)
        f.close()
    print("file write complete\n")
    
with open("qa_list_all.txt","r") as f:
    qa_all=json.load(f)

qa=qa_all[0]
print(qa["company"])

produce_all(qa_all,glove)




<>:17: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:17: SyntaxWarning: "is not" with a literal. Did you mean "!="?
C:\Users\ly0001\AppData\Local\Temp\ipykernel_1856\1197571992.py:17: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if line[0] is not '<':


Loading Glove Model


KeyboardInterrupt: 

In [28]:
def compile_table(qa_all, glove):
    table = []
    for earning_report in qa_all:
        company = earning_report["company"]
        year = earning_report["year"]
        quarter = earning_report["quarter"]
        for qa in earning_report["QA_session"]:
            for session in qa:
                q_company = session["Q_company_name"]
                q_analyst = session["Q_analyst_name"]
                similarity = get_similarity1(session, glove)
                if similarity != [0]:
                    table.append([q_analyst, q_company, similarity])
    return table

table = compile_table(qa_all, glove)


for row in table:
    row[2] = round(row[2], 3)

# Move the rows with empty analyst and company to the end of table
table.sort(key=lambda row: (not row[0], not row[1]))

TypeError: type list doesn't define __round__ method

In [26]:
table

[['Edward White', 'Lehman', [0.030466267095893174]],
 ['Edward White', 'Lehman', [0.12163109908097136]],
 [' Dean Gray', 'Goldman Sachs.', [0.02800564676385109, 0.0210829570844574]],
 [' Dean Gray', 'Goldman Sachs.', [0.06759445407484066]],
 [' Dean Gray',
  'Goldman Sachs.',
  [0.2256850548698165, 0.041877262449731556, 0.07486162386716366]],
 ['', '', [0.021829645282110843]],
 [' Richard Shoe', 'SG Cowen.', [0.16107416534397234]],
 [' Richard Shoe', 'SG Cowen.', [0.39433198226342436]],
 [' Richard Shoe', 'SG Cowen.', [0.3816315287943527]],
 [' Richard Shoe', 'SG Cowen.', [0.266693716241618]],
 [' Richard Shoe', 'SG Cowen.', [0.08344411607395585]],
 [' Richard Shoe', 'SG Cowen.', [0.12028533381165507]],
 [' Richard Shoe', 'SG Cowen.', [0.04965384436835463]],
 ['', '', [0.0715518492151519]],
 [' Alec Berming', '[indiscernible] Capital.', [0.04015792060302614]],
 [' Paul [indiscernible]',
  'Thomas Weisel Partners.',
  [0.07843705765968623, 0.029785291702925987]],
 [' Dean Gray', 'Goldma